## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:32,  8.05it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [01:34<00:00, 79.56it/s] 


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Shashi Tharoor,New Delhi & Thiruvananthapuram,MP for Thiruvananthapuram. Author of 20 books....,2009-03-16 15:12:51,7646971,989,18727,True,2020-08-08 14:43:07,Don't know whether to laugh or cry! Polls are ...,None,Twitter Web App,False
0,Journalist Siraj Noorani,mumbai,Journalist @hathodapost & @4tvhyd Secretary At...,2010-08-22 10:03:30,635,2682,3386,False,2020-08-08 14:43:06,"5,883 new #COVID19 positive cases and 118 deat...","[COVID19, TamilNadu]",Twitter for Android,False
0,André Picard,Montréal,André Picard is the health columnist at The Gl...,2011-02-15 22:51:18,112462,1809,1644,True,2020-08-08 14:43:00,Why the #Coronavirus is More Likely to ‘Supers...,[Coronavirus],Twitter Web App,False
0,NYC Immigrant Affairs,New York City,Where you were born does not matter—when you'r...,2010-02-19 14:00:39,23836,1308,5503,True,2020-08-08 14:43:00,The #COVID19 fight will only be won if we work...,[COVID19],Twitter Media Studio,False
0,Offeh Michael,Ghana,Physician Assistant | Global Public Health | T...,2013-06-12 22:51:51,207,871,1380,False,2020-08-08 14:42:57,Wear a mask. \nSanitize ot wash yout hands fre...,[WearAMaskChallenge],Twitter for Android,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (82710, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 82710 all tweets: 90210


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (90210, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)